# API Example(Banking, Trading, Machine Learning...)

## mongoDB, AWS, Docker

```shell
mkdir web
mkdir db

touch web/Dockerfile
touch db/Dockerfile
cd web/

touch app.py
touch requirements.txt


```

- root /docker_compose.yml

```yml
version: '3'
services:
  web:
    build: ./web
    ports:
      - "5000:5000"
    links:
      - db
  db:
    build: ./db


```

- db/Dockerfile

```python
FROM mongo:3.6.4

```

- web/Dockerfile  

    - "./" menas "/usr/src/app"  
    
```python
FROM python:3
WORKDIR /usr/src/app
COPY requirements.txt ./  
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD [ "python", "app.py" ]

```

- web/requirements.txt

```python
Flask
flask_restful
pymongo
bcrypt
```

- add python library 

```python
pandas== version?
numpy== version?
matplotlib
h5py
glob3
sklearn
scikit-optimize
finance_datareader?
yfinance
pandas_datareader?
tensorflow==1.14?
keras
```


```python

# flask , flask_restful, pymongo, bcrypt library
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
from pymongo import MongoClient
import bcrypt


# Create Flask object
# Create flask_restful object with Flask object input
app = Flask(__name__)
# API constructor
api = Api(app)

# Create MongoClient object
# Create new mongo db MoneyManagementDB
# Create new collections users
client = MongoClient("mongodb://db:27017")
db = client.MoneyManagementDB
users = db["Users"]

# check user name in users collections of MoneyManagementDB
def UserExist(username):
    if users.find({"Username":username}).count() == 0:
        return False
    else:
        return True


# Create Register resource
# inherit Flask restful API resource
class Register(Resource):

    # post method
    def post(self):
        # from flask request.get_json()
        #Step 1 is to get posted data by the user
        postedData = request.get_json()

        #Get the data
        username = postedData["username"]
        password = postedData["password"] #"123xyz"

        # if user exists in db, 301 status returns
        if UserExist(username):
            retJson = {
                'status':301,
                'msg': 'Invalid Username'
            }
            return jsonify(retJson)

        # password encrypted
        hashed_pw = bcrypt.hashpw(password.encode('utf8'), bcrypt.gensalt())

        # user collections inserts the new person in users collections
        #Store username and pw into the database
        users.insert({
            "Username": username,
            "Password": hashed_pw,
            "Own":0,
            "Debt":0
        })

        retJson = {
            "status": 200,
            "msg": "You successfully signed up for the API"
        }
        return jsonify(retJson)
    
```

- helper function 

- DB columns: 

```
    "Username"
    "Password"
    "Own"
    "Debt"
```

```python
# verify password , compare hashed_pw from db and password from json
def verifyPw(username, password):
    if not UserExist(username):
        return False

    hashed_pw = users.find({
        "Username":username
    })[0]["Password"]

    if bcrypt.hashpw(password.encode('utf8'), hashed_pw) == hashed_pw:
        return True
    else:
        return False

# return user's cash in user's account
def cashWithUser(username):
    cash = users.find({
        "Username":username
    })[0]["Own"]
    return cash

# return debt from user['Username'][0]['Debt']
def debtWithUser(username):
    debt = users.find({
        "Username":username
    })[0]["Debt"]
    return debt

def generateReturnDictionary(status, msg):
    retJson = {
        "status": status,
        "msg": msg
    }
    return retJson

# return Error msg,True/False
def verifyCredentials(username, password):
    if not UserExist(username):
        return generateReturnDictionary(301, "Invalid Username"), True

    correct_pw = verifyPw(username, password)

    if not correct_pw:
        return generateReturnDictionary(302, "Incorrect Password"), True

    return None, False



# mongodb collection_name.update method
# update mongo db Own field in user collections
def updateAccount(username, balance):
    users.update({"Username": username},{ "$set":{"Own": balance} } )
    
    

```



- ADD flask resource API

```python
# Create ADD resource
class Add(Resource):
    # post method
    def post(self):
        # from flask import Flask, jsonify, request
        # from flask_restful import Api, Resource
        postedData = request.get_json()

        username = postedData["username"]
        password = postedData["password"]
        money = postedData["amount"]

        retJson, error = verifyCredentials(username, password)
        if error:
            return jsonify(retJson)

        # check if the money is less than zero,
        if money<=0:
            return jsonify(generateReturnDictionary(304, "The money amount entered must be greater than 0"))


        cash = cashWithUser(username)
        money-= 1 #Transaction fee

        # bank has its own BANK db collections
        #Add transaction fee to bank account
        # "BANK" is user name in mongoDB
        bank_cash = cashWithUser("BANK")
        updateAccount("BANK", bank_cash+1)

        # update the user's own field
        #Add remaining to user
        updateAccount(username, cash+money)

        return jsonify(generateReturnDictionary(200, "Amount Added Successfully to account"))


```

```python
# Create Transfer resource
class Transfer(Resource):
    def post(self):
        postedData = request.get_json()

        username = postedData["username"]
        password = postedData["password"]
        to       = postedData["to"]
        money    = postedData["amount"]


        retJson, error = verifyCredentials(username, password)
        if error:
            return jsonify(retJson)

        cash = cashWithUser(username)
        if cash <= 0:
            return jsonify(generateReturnDictionary(303, "You are out of money, please Add Cash or take a loan"))

        if money<=0:
            return jsonify(generateReturnDictionary(304, "The money amount entered must be greater than 0"))

        if not UserExist(to):
            return jsonify(generateReturnDictionary(301, "Recieved username is invalid"))

        # we need thress db information, sender, receiver, bank
        cash_from = cashWithUser(username)
        cash_to   = cashWithUser(to)
        bank_cash = cashWithUser("BANK")

        updateAccount("BANK", bank_cash+1)
        updateAccount(to, cash_to+money-1)
        updateAccount(username, cash_from - money)

        retJson = {
            "status":200,
            "msg": "Amount added successfully to account"
        }
        return jsonify(generateReturnDictionary(200, "Amount added successfully to account"))
```


```python
class TakeLoan(Resource):
    def post(self):
        postedData = request.get_json()

        username = postedData["username"]
        password = postedData["password"]
        money    = postedData["amount"]

        retJson, error = verifyCredentials(username, password)
        if error:
            return jsonify(retJson)

        cash = cashWithUser(username)
        debt = debtWithUser(username)
        updateAccount(username, cash+money)
        updateDebt(username, debt + money)

        return jsonify(generateReturnDictionary(200, "Loan Added to Your Account"))

class PayLoan(Resource):
    def post(self):
        postedData = request.get_json()

        username = postedData["username"]
        password = postedData["password"]
        money    = postedData["amount"]

        retJson, error = verifyCredentials(username, password)
        if error:
            return jsonify(retJson)

        cash = cashWithUser(username)

        if cash < money:
            return jsonify(generateReturnDictionary(303, "Not Enough Cash in your account"))

        debt = debtWithUser(username)
        updateAccount(username, cash-money)
        updateDebt(username, debt - money)

        return jsonify(generateReturnDictionary(200, "Loan Paid"))


```



```python
# app = Flask(__name__)
# api = Api(app)


api.add_resource(Register, '/register')
api.add_resource(Add, '/add')
api.add_resource(Transfer, '/transfer')
api.add_resource(Balance, '/balance')
api.add_resource(TakeLoan, '/takeloan')
api.add_resource(PayLoan, '/payloan')


if __name__=="__main__":
    app.run(host='0.0.0.0')
```

## Asynchronous restAPI

[Flask asynchronous background tasks with Celery and Redis](http://allynh.com/blog/flask-asynchronous-background-tasks-with-celery-and-redis/)  

[Asynchronous updates to a webpage with Flask and Socket.io](https://www.shanelynn.ie/asynchronous-updates-to-a-webpage-with-flask-and-socket-io/)  

[Testing Asynchronous APIs with Postman](https://medium.com/@devdoneuk/testing-asynchronous-apis-with-postman-8ef56b125ba)  

[Postman Test examples](https://learning.getpostman.com/docs/postman/scripts/test_examples/)  


